In [18]:
import os
import json
import pandas as pd
from IPython.display import display, HTML


def load_results(results_dir="results"):
  """
  Load results from all subdirectories in the specified results directory.
  """
  data = []
  for folder in os.listdir(results_dir):
    folder_path = os.path.join(results_dir, folder)
    if os.path.isdir(folder_path):
      try:
        with open(os.path.join(folder_path, "config.json"), "r") as f:
          config = json.load(f)
        with open(os.path.join(folder_path, "results.json"), "r") as f:
          results = json.load(f)
        with open(os.path.join(folder_path, "best_episode.json"), "r") as f:
          best_episode = json.load(f)

        data.append({
            "Model": config["model"].upper(),
            "Environment": config.get("environment", "N/A"),
            "Episodes": config["episodes"],
            "Average Episode Time": round(results.get("average_time", 0), 2),
            "Reward Strategy": config.get("reward_strategy", "default"),
            "Average Reward": results.get("average_reward", 0),
            "Reward Variance": results.get("variance_in_rewards", 0),
            "Best Episode Reward": best_episode.get("reward", 0),
            "Results Folder": folder_path
        })
      except (FileNotFoundError, KeyError) as e:
        print(f"Skipping folder {folder_path} due to missing or invalid files.")
  return data


def display_results_table(data):
  """
  Display the results data as a sortable table in a Jupyter Notebook.
  """
  df = pd.DataFrame(data)
  df = df.sort_values(by=["Average Reward", "Reward Variance"], ascending=[False, True])
  display(HTML(df.to_html(index=False, escape=False, classes="sortable", border=1)))


# Load and display results
results_data = load_results("results")

# sort on Best Episode Reward
results_data = sorted(results_data, key=lambda x: x["Best Episode Reward"], reverse=True)

display_results_table(results_data)

Skipping folder results/TOTD-20241124-164847 due to missing or invalid files.
Skipping folder results/mac-20241124-163611 due to missing or invalid files.
Skipping folder results/totd-20241124-163610 due to missing or invalid files.
Skipping folder results/laber-20241124-163610 due to missing or invalid files.
Skipping folder results/ppo-20241124-163611 due to missing or invalid files.
Skipping folder results/ddpg-20241124-163611 due to missing or invalid files.
Skipping folder results/TRPO-20241124-164739 due to missing or invalid files.
Skipping folder results/sac-20241124-163611 due to missing or invalid files.
Skipping folder results/TOTD-20241124-165821 due to missing or invalid files.
Skipping folder results/totd-20241124-163611 due to missing or invalid files.
Skipping folder results/trpo-20241124-163611 due to missing or invalid files.
Skipping folder results/PPO-20241124-165741 due to missing or invalid files.
Skipping folder results/ppo-20241124-163610 due to missing or inval

Model,Environment,Episodes,Average Episode Time,Reward Strategy,Average Reward,Reward Variance,Best Episode Reward,Results Folder
TOTD,LunarLanderContinuous-v3,10,4.70,default,0.000000,0.000000,0.000000,results/TOTD-20241124-164652
TRPO,LunarLanderContinuous-v3,5,1.10,default,-95.011921,1562.870199,-47.552824,results/TRPO-20241124-162837
TRPO,LunarLanderContinuous-v3,5,0.82,default,-114.675348,3559.941229,-42.222788,results/TRPO-20241124-163021
TOTD,LunarLanderContinuous-v3,5,1.20,default,-150.066956,7710.044077,-63.350067,results/TOTD-20241124-163143
MAC,LunarLanderContinuous-v3,5,0.11,default,-150.726576,6869.446292,-81.909330,results/MAC-20241124-163140
LABER,LunarLanderContinuous-v3,10,1.38,default,-160.508233,7037.351783,-14.520676,results/LABER-20241124-164152
TRPO,LunarLanderContinuous-v3,10,11.95,default,-163.412542,26046.116226,20.010954,results/TRPO-20241124-165508
TOTD,LunarLanderContinuous-v3,10,5.36,default,-164.541619,16161.676557,-44.221669,results/TOTD-20241124-165415
PPO,LunarLanderContinuous-v3,5,1.17,default,-189.252350,12132.747160,-66.180677,results/PPO-20241124-163452
MAC,LunarLanderContinuous-v3,10,1.04,default,-190.272354,38833.121974,22.276983,results/MAC-20241124-165731
